In [ ]:
using ProgressMeter, PyCall, PyPlot, Images, HDF5,NaNStatistics, Statistics, DSP, Lasso, JLD2
using _Data, _Math

In [ ]:
@pyimport numpy

In [ ]:
rc_params = PyDict(pyimport("matplotlib")["rcParams"]);
# rc_params["font.sans-serif"] = ["Arial"];
rc_params["font.size"] = 7;
rc_params["lines.linewidth"] = 1;
rc_params["lines.markersize"] = 4;
rc_params["xtick.major.size"] = 2;
rc_params["ytick.major.size"] = 2;
rc_params["xtick.major.pad"] = 2;
rc_params["ytick.major.pad"] = 2;
# rc_params["axes.labelpad"] = 2;

cim(img::Matrix{UInt32}) = CairoImageSurface(img, Cairo.FORMAT_RGB24; flipxy = false) 
cim(img::Matrix{ARGB32}) = cim(reinterpret(UInt32, img))
cim(img::Matrix{RGB24}) = cim(reinterpret(UInt32, img))
cim(img::Matrix{UInt8}) = cim(Gray.(reinterpret(N0f8, img)))
cim(img::Array{UInt8,3}) = cim(RGB24.(reinterpret(N0f8, img[:,:,1]), reinterpret(N0f8, img[:,:,2]), reinterpret(N0f8, img[:,:,3])));downsample(img,n=4) = +([img[i:n:n*div(size(img,1)-1,n), j:n:n*div(size(img,2)-1,n)] for i = 1:n, j = 1:n]...)/(n*n);
downsample(img,n=4) = +([img[i:n:n*div(size(img,1)-1,n), j:n:n*div(size(img,2)-1,n)] for i = 1:n, j = 1:n]...)/(n*n);

In [ ]:
fig_path = "/home/chuyu/Notebooks/project_place_cell/figures/output/review_figures/ABA/"
mkpath(fig_path)

In [ ]:
include("../../../functions/func_map.jl")
include("../../../functions/func_stat.jl")
include("../../../functions/func_data.jl")
include("../../../functions/func_plot.jl")

In [ ]:
data_info_all = [
    ["20240128_132056", 1, "20240128_152133", 1, "20240128_164754", 1, "drew"],
    ["20240127_185226", 8, "20240127_202210", 8, "20240127_212910", 8, "jen"],
    ["20240127_120909_20240127_132316", 1, "20240127_152329", 1, "20240127_164645", 1, "chuyu"],
    ["20240128_174842_20240128_182823", 8, "20240128_194250", 8, "20240128_203533", 8, "chuyu"],

    ]

In [ ]:
which_data = 4
data_info = data_info_all[which_data]
experiment_filename_1 = data_info[1]
server_1 = data_info[2]

experiment_filename_2 = data_info[3]
server_2 = data_info[4]

experiment_filename_3 = data_info[5]
server_3 = data_info[6]

experimenter = data_info[end]
analyzer = "chuyu"

In [ ]:
ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")

ds_save_2 = Dataset(experiment_filename_2, experimenter, gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)") # This example dataset is on roli-9, so the path is different depending on whether you're trying to access the file from roli-9
ds_save_cy_2 = Dataset(experiment_filename_2, "chuyu", gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)")
ds_save_analyzer_2 = Dataset(experiment_filename_2, analyzer, gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)")

ds_save_3 = Dataset(experiment_filename_3, experimenter, gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)") # This example dataset is on roli-9, so the path is different depending on whether you're trying to access the file from roli-9
ds_save_cy_3 = Dataset(experiment_filename_3, "chuyu", gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)")
ds_save_analyzer_3 = Dataset(experiment_filename_3, analyzer, gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)")

In [ ]:
# orientation-corrected background image and chamber roi image
orientation_correction_file = h5open(joinpath(data_path(ds_save_cy_1), "chamber_geometry_$(experiment_filename_1).h5"))
chamber_roi_1 = read(orientation_correction_file,"chamber_roi")
countour_array_1 = read(orientation_correction_file,"countour_array")
center_loc_1 = read(orientation_correction_file,"center_loc")
close(orientation_correction_file)


# orientation-corrected background image and chamber roi image
orientation_correction_file = h5open(joinpath(data_path(ds_save_cy_2), "chamber_geometry_$(experiment_filename_2).h5"))
chamber_roi_2 = read(orientation_correction_file,"chamber_roi")
countour_array_2 = read(orientation_correction_file,"countour_array")
center_loc_2 = read(orientation_correction_file,"center_loc")
close(orientation_correction_file)


In [ ]:
experiment_filename_replace = "20240123_164218"
server_replace = 9

ds_save_replace = Dataset(experiment_filename_replace, "chuyu", gethostname() == "roli-$(server_replace)" ? "/data" : "/nfs/data$(server_replace)")

# orientation-corrected background image and chamber roi image
orientation_correction_file = h5open(joinpath(data_path(ds_save_replace), "chamber_geometry_$(experiment_filename_replace).h5"))
chamber_roi_2 = read(orientation_correction_file,"chamber_roi")
countour_array_2 = read(orientation_correction_file,"countour_array")
center_loc_2 = read(orientation_correction_file,"center_loc")
close(orientation_correction_file)

In [ ]:
for_place_calculation_file = h5open(joinpath(data_path(ds_save_cy_1), "for_place_calculation_chamber_geometry_$(experiment_filename_1)_n90.h5"))
x_fish_sweep_mean_1 = read(for_place_calculation_file,"x_fish_sweep_mean")
y_fish_sweep_mean_1 = read(for_place_calculation_file,"y_fish_sweep_mean")
close(for_place_calculation_file)


for_place_calculation_file = h5open(joinpath(data_path(ds_save_cy_2), "for_place_calculation_chamber_geometry_$(experiment_filename_2)_n90.h5"))
x_fish_sweep_mean_2 = read(for_place_calculation_file,"x_fish_sweep_mean")
y_fish_sweep_mean_2 = read(for_place_calculation_file,"y_fish_sweep_mean")
close(for_place_calculation_file)


for_place_calculation_file = h5open(joinpath(data_path(ds_save_cy_3), "for_place_calculation_chamber_geometry_$(experiment_filename_3)_n90.h5"))
x_fish_sweep_mean_3 = read(for_place_calculation_file,"x_fish_sweep_mean")
y_fish_sweep_mean_3 = read(for_place_calculation_file,"y_fish_sweep_mean")
close(for_place_calculation_file)

In [ ]:
# peak locations

chamber_roi_mask = Float32.(copy(chamber_roi_1))
chamber_roi_mask[chamber_roi_mask.!=0].= NaN32
valid_frame = isnan.(chamber_roi_mask[:,:,1]); # bs update
ymin = findfirst(sum(valid_frame;dims=1).!=0)[2]-10; # bs update
ymax = findlast(sum(valid_frame;dims=1).!=0)[2]+10; # bs update
xmin = findfirst(sum(valid_frame;dims=2).!=0)[1]-10; # bs update
xmax = findlast(sum(valid_frame;dims=2).!=0)[1]+10; # bs update

countour_array = countour_array_1

fig = figure()

plot(x_fish_sweep_mean_1, y_fish_sweep_mean_1, color="k")

axis("off")
axis("equal")
# plot(vcat(countour_array[1:50:end,1][end], countour_array[1:50:end,1]),vcat(countour_array[1:50:end,2][end], countour_array[1:50:end,2]),c="k", alpha=0.1)

xlim(xmin,xmax)
ylim(ymax,ymin)
fig.savefig(joinpath(fig_path, "A1_trajectory.png"), bbox_inches = "tight",transparent = true,pad_inches = 0)



# peak locations

chamber_roi_mask = Float32.(copy(chamber_roi_2))
chamber_roi_mask[chamber_roi_mask.!=0].= NaN32
valid_frame = isnan.(chamber_roi_mask[:,:,1]); # bs update
ymin = findfirst(sum(valid_frame;dims=1).!=0)[2]-10; # bs update
ymax = findlast(sum(valid_frame;dims=1).!=0)[2]+10; # bs update
xmin = findfirst(sum(valid_frame;dims=2).!=0)[1]-10; # bs update
xmax = findlast(sum(valid_frame;dims=2).!=0)[1]+10; # bs update

countour_array = countour_array_2

fig = figure()

plot(x_fish_sweep_mean_2, y_fish_sweep_mean_2, color="k")

axis("off")
axis("equal")
# plot(vcat(countour_array[1:50:end,1][end], countour_array[1:50:end,1]),vcat(countour_array[1:50:end,2][end], countour_array[1:50:end,2]),c="k", alpha=0.1)

xlim(xmin,xmax)
ylim(ymax,ymin)
fig.savefig(joinpath(fig_path, "B_trajectory.png"), bbox_inches = "tight",transparent = true,pad_inches = 0)



# peak locations

chamber_roi_mask = Float32.(copy(chamber_roi_1))
chamber_roi_mask[chamber_roi_mask.!=0].= NaN32
valid_frame = isnan.(chamber_roi_mask[:,:,1]); # bs update
ymin = findfirst(sum(valid_frame;dims=1).!=0)[2]-10; # bs update
ymax = findlast(sum(valid_frame;dims=1).!=0)[2]+10; # bs update
xmin = findfirst(sum(valid_frame;dims=2).!=0)[1]-10; # bs update
xmax = findlast(sum(valid_frame;dims=2).!=0)[1]+10; # bs update

countour_array = countour_array_1

fig = figure()

plot(x_fish_sweep_mean_3, y_fish_sweep_mean_3, color="k")

axis("off")
axis("equal")
# plot(vcat(countour_array[1:50:end,1][end], countour_array[1:50:end,1]),vcat(countour_array[1:50:end,2][end], countour_array[1:50:end,2]),c="k", alpha=0.1)

xlim(xmin,xmax)
ylim(ymax,ymin)
fig.savefig(joinpath(fig_path, "A2_trajectory.png"), bbox_inches = "tight",transparent = true,pad_inches = 0)